# 第5回：AIの社会実装（Webアプリ開発）

## 学習目標
- 学習したモデルを**インタラクティブなWebアプリとして公開**する
- **Gradio**を用いて、誰でもブラウザからAI診断を体験できるようにする

## Gradioとは？
Gradioは、機械学習モデルを簡単にWebアプリ化できるPythonライブラリです。
- 数行のコードでUIを構築
- `share=True` で一時的な公開URLを発行可能（72時間有効）
- 友人や同僚に「自分が作ったAI」を共有できる

---
## はじめに

**操作方法**: `Shift + Enter` でセルを順番に実行してください。セッションが切断された場合は、上から再実行すれば復旧できます。

---

### この講義で挑戦すること

優れたAIモデルを作っても、Jupyter Notebookの中だけで動くなら、それは「研究室の中」に閉じ込められているのと同じです。医師や患者が使えるようにするには、**誰でも簡単に試せる形**にする必要があります。

この講義では、画像分類AIを**Webアプリ**として公開します。URLを共有するだけで、世界中の誰もがあなたのAIを試せるようになります。AIを「作る」から「届ける」へ、一歩踏み出しましょう。

---
## なぜ社会実装が重要か

優れたAIモデルを開発しても、ユーザーが使える形で提供しなければ価値を発揮できません。Webアプリ化することで、プログラミングの知識がない人でもAIを活用できるようになります。

料理のレシピを開発しても、それを料理本やアプリで公開しなければ誰も作れません。同様に、AIモデルも「使える形」にして初めて社会の役に立ちます。Gradioは、その「公開」を数行のコードで実現するツールです。

---
## Step 1: 環境構築

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise.git /tmp/MedMNIST-Exercise -q
sys.path.insert(0, '/tmp/MedMNIST-Exercise')
sys.modules.pop('exercise_logic', None)
import exercise_logic
exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")


---
## Step 2: データセットの選択

第1回〜第4回で扱ったデータセットから1つを選んでください。

| データセット | 内容 | クラス数 |
|-------------|------|----------|
| pathmnist | 病理画像 | 9クラス |
| dermamnist | 皮膚病変 | 7クラス |
| chestmnist | 胸部X線 | 2クラス（正常/異常） |

In [ ]:
# ============================================
# 使用するデータセットを1つ選んでください
# 使いたい行のコメント(#)を外してください
# ============================================

data_flag = 'pathmnist'      # 第1回: 病理画像（9クラス分類）
# data_flag = 'dermamnist'   # 第2回: 皮膚病変（7クラス分類）
# data_flag = 'chestmnist'   # 第3回・第4回: 胸部X線（2値分類）

---
## Step 3: データ読み込みとモデル学習

選択したデータセットでモデルを学習します。

In [ ]:
# データの読み込み
# chestmnistの場合は2値分類に変換
if data_flag == 'chestmnist':
    (x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess(
        data_flag, 
        binary_classification=True
    )
    num_classes = 1
else:
    (x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess(data_flag)
    num_classes = len(info['label'])

print(f"データセット: {data_flag}")
print(f"訓練データ: {x_train.shape}")
print(f"ラベル一覧: {info['label']}")

In [ ]:
# モデルの構築と学習
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=num_classes,
    model_type='simple'
)

print("モデルを学習中...（約2-3分）")
history = model.fit(
    x_train, y_train, 
    epochs=5, 
    validation_split=0.1, 
    batch_size=128,
    verbose=1
)
print("\n✓ モデル学習が完了しました。")

---
## Step 4: Gradio Webアプリの作成

学習したモデルをWebアプリとして公開します。

**機能:**
- 画像をアップロード
- AIが診断結果を予測
- 各クラスの確率を棒グラフで表示

In [ ]:
# Gradioアプリの構築
demo = exercise_logic.create_gradio_app(model, info, data_flag)

print("✓ Gradioアプリの準備が完了しました。")
print("次のセルを実行してアプリを起動してください。")

---
## Step 5: サンプル画像のダウンロード

- Webアプリで試すためのサンプル画像をダウンロードします
- テストデータから各クラス最低1枚を含む10枚の画像を選んでJPEGファイルとして保存します
- 実行するたびに異なるサンプルがランダムに選ばれます

In [ ]:
# サンプル画像の作成
zip_path = exercise_logic.create_sample_images(x_test, y_test, info, data_flag, n_samples=10)

print("次のセルを実行してダウンロードしてください。")

In [ ]:
# サンプル画像のダウンロード（Google Colab専用）
try:
    from google.colab import files
    files.download('/tmp/sample_images.zip')
    print("✓ ダウンロードが開始されました。")
    print("ダウンロードしたZIPファイルを解凍して、Webアプリで試してください。")
except ImportError:
    print("Google Colab以外の環境では、以下のパスからファイルを取得してください:")
    print(f"  {zip_path}")

---
## Step 6: アプリの起動

以下のセルを実行すると、Webアプリが起動します。

- **ローカルURL**: Colab内で直接使用可能
- **公開URL**: `share=True` により、72時間有効なURLが発行されます。このURLを共有すれば、誰でもあなたのAIを試すことができます。

**試し方:**
1. 上でダウンロードしたサンプル画像を解凍
2. Webアプリの「Upload Image」をクリック
3. サンプル画像を選択してアップロード
4. AIの予測結果を確認

In [ ]:
# アプリの起動（公開URLを発行）
demo.launch(share=True)

---
## 演習課題

### 課題1: 異なるデータセットで試す
Step 2でデータセットを変更し、再度全てのセルを実行してみましょう。

### 課題2: 考察
以下の点について考えてみましょう：

1. WebアプリとしてAIを公開することで、どのようなメリットがありますか？
2. 医療AIをWebアプリとして公開する場合、どのような注意点がありますか？
3. 実際の医療現場でAIを導入する際、どのような課題があると思いますか？

---
## まとめ

- **Gradio**: 機械学習モデルを簡単にWebアプリ化できるライブラリ
- **share=True**: 一時的な公開URLを発行し、誰でもアクセス可能に
- **AIの社会実装**: モデルを作るだけでなく、ユーザーが使える形で提供することが重要

### 全5回のまとめ

| 回 | テーマ | 学んだこと |
|----|--------|------------|
| 第1回 | CNN入門 | 畳み込みニューラルネットワークによる画像分類 |
| 第2回 | 転移学習 | 学習済みモデルの活用、クラス不均衡への対処 |
| 第3回 | Grad-CAM | AIの判断根拠の可視化、説明可能なAI |
| 第4回 | 評価指標 | ROC曲線・AUC、感度・特異度と閾値設定 |
| 第5回 | 社会実装 | Webアプリ化、AIの公開と共有 |

---
## 考察課題の回答例

以下は考察課題に対する回答の一例です。これが唯一の正解ではなく、議論の出発点として活用してください。

### 1. Webアプリ公開のメリット

AIをWebアプリとして提供する利点：

- 利用者側の環境構築負荷を小さくできる（ブラウザで利用可能）
- 入力→結果までが短く、フィードバックが得やすい
- URL共有で試用・評価が容易
- サーバー側で更新でき、改善を配布しやすい（ただし変更管理と性能監視が重要）

### 2. 医療AIをWebアプリで提供する際の注意点

医療分野では特に以下が重要：

- **規制**: 医療目的のソフトは医療機器プログラム（SaMD）に該当し得るため、該当性判断と要求事項の整理が必要
- **プライバシー**: 患者画像の取り扱い、同意、保管、アクセス制御、ログ管理などを厳格に
- **セキュリティ**: 暗号化、脆弱性対応、権限管理、サイバー対策
- **表示と運用**: 意図された使用、適用範囲、限界、想定外使用への警告、過信防止の設計

### 3. 医療現場でのAI導入の課題

導入・運用で問題になりやすい点：

- 医療従事者の受容と教育（使い方・限界の理解）
- 既存ワークフロー（PACS/EHR等）との統合
- 責任分界とインシデント対応
- 導入・運用・保守コスト
- 継続的な性能監視（データドリフト、施設差、時間変化）と変更管理

---
## 発展的な学習（技術的詳細に興味のある方へ）

この講義では、技術的な詳細を `exercise_logic.py` に分離しています。
より深く学びたい方は、以下の関数のソースコードを参照してください。

### この講義で使用した主要関数

| 関数名 | 機能 | 技術的なポイント |
|--------|------|------------------|
| `initialize_environment()` | 環境セットアップ | Colab/Local判定、GPU設定 |
| `load_and_preprocess()` | データ読み込み・前処理 | 複数データセットへの対応 |
| `build_model()` | モデル構築 | simple/transferの切り替え |
| `create_gradio_app()` | Webアプリ構築 | gr.Interfaceの構成、predict関数の実装 |
| `create_sample_images()` | サンプル画像生成 | ハイブリッドサンプリング、ZIP圧縮 |

### ソースコードの参照方法

`exercise_logic.py` はGitHubリポジトリで公開しています：

https://github.com/kshimoji8/MedMNIST-Exercise/blob/main/exercise_logic.py

各関数には詳細な技術解説をdocstring（関数冒頭のコメント）として記載しています。